In [4]:
import numpy as np

In [5]:
M = 20  # kg
g = 9.8 # m / s^2
m = 0.5 # kg
l = 0.5 # m

In [6]:
v1 = ((M + m) * g) / (M * l)
v2 = -(m * g) / M

c1 = -1 / (M * l)
c2 = 1 / M

In [7]:
A = np.array([[0, 1, 0, 0], [0, 0, v2, 0], [0, 0, 0, 1], [0, 0, v1, 0]])
B = np.array([[0], [c2], [0], [c1]])

In [8]:
# print A and B
print("A = \n", A)
print("B = \n", B)

A = 
 [[ 0.     1.     0.     0.   ]
 [ 0.     0.    -0.245  0.   ]
 [ 0.     0.     0.     1.   ]
 [ 0.     0.    20.09   0.   ]]
B = 
 [[ 0.  ]
 [ 0.05]
 [ 0.  ]
 [-0.1 ]]


In [21]:
def loss(inputs):
    loss = []
    desired_output = np.array([0, 0, 0, 0])
    for i in range(np.shape(inputs)[0]):
        x_curr = inputs[i]
        var = ((x_curr[0] - desired_output[0])**2)  + ((x_curr[1] - desired_output[1])**2)  + ((x_curr[2] - desired_output[2]) % (2 * np.pi))**2 + (x_curr[3] - desired_output[3])**2
        loss.append(var)
        # print(var)

    loss = np.array(loss)
    fin_loss = np.mean(loss, axis=0)
    print(fin_loss)
    return np.mean(loss, axis=0)
        
def next_state(A, B, x_curr, u_curr, dt):
    x_dot = A @ x_curr + B @ u_curr
    x_next = np.copy(x_dot)
    
    x_next[0] = x_curr[0] - x_dot[0] * dt
    x_next[1] = x_curr[1] + x_dot[1] * dt
    x_next[2] = x_curr[2] + x_dot[2] * dt
    x_next[3] = x_curr[3] + x_dot[3] * dt

    return x_next

## NN

In [22]:
# Define the activation function (sigmoid)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Define the derivative of the activation function (sigmoid)
def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

# Define the number of input, hidden, and output nodes
input_nodes = 4
hidden_nodes = 8
output_nodes = 1

global input_weights
global hidden_weights
global hidden_bias
global output_bias
global learning_rate

learning_rate = 0.01

# Initialize the weights for the input and hidden layers
input_weights = np.zeros((input_nodes, hidden_nodes)).reshape(4, 8)
hidden_weights = np.zeros((hidden_nodes, output_nodes))

# Initialize the biases for the hidden and output layers
hidden_bias = np.zeros((1, hidden_nodes))
output_bias = np.zeros((1, output_nodes))

# Define the forward propagation function
def forward_propagation(inputs):
    hidden_layer = sigmoid(np.dot(inputs.T, input_weights) + hidden_bias)
    output_layer = sigmoid(np.dot(hidden_layer, hidden_weights) + output_bias)
    return output_layer, hidden_layer

# Define the backward propagation function
def backward_propagation(inputs, outputs, hidden_layer, hidden_weights, input_weights, hidden_bias, output_bias):
    error = loss(inputs)
    # print(error)
    output_delta = error * sigmoid_derivative(outputs).reshape(20, 1)
    hidden_error = output_delta.dot(hidden_weights.T)
    hidden_delta = hidden_error * sigmoid_derivative(hidden_layer)
    hidden_weights += hidden_layer.T.dot(output_delta) * learning_rate
    # print(np.shape(input_weights), np.shape(inputs))
    inputs = inputs.reshape(20, 4)
    input_weights += inputs.T.dot(hidden_delta) * learning_rate
    output_bias += np.sum(output_delta, axis=0, keepdims=True) * learning_rate
    hidden_bias += np.sum(hidden_delta, axis=0, keepdims=True) * learning_rate

# Define the training function
# def train(inputs, learning_rate, epochs):
#     for i in range(epochs):
#         output_layer, hidden_layer = forward_propagation(inputs)
#         backward_propagation(inputs, output_layer, hidden_layer, hidden_weights, input_weights, hidden_bias, output_bias)
#         if i % 200 == 0:
#             print("Epoch", i, "Loss", np.mean(np.abs(np.array([10.63, 0, 0, 0]) - output_layer)))

# Generating inputs and outputs
for k in range(20):
    initial_input = np.array([[0], [0], [np.pi/6], [0]])
    inputs = [initial_input]

    initial_output, hidden_layer =  forward_propagation(initial_input)
    outputs = [initial_output]
    hidden_outputs = [hidden_layer]

    for i in range(19):
        x_curr = inputs[i]
        u_curr = outputs[i]
        x_next = next_state(A, B, x_curr, u_curr, dt = 0.1)
        inputs.append(x_next)
        output_layer, hidden_layer = forward_propagation(x_next)
        outputs.append(output_layer)
        hidden_outputs.append(hidden_layer)

    inputs = np.array(inputs)
    outputs = np.array(outputs)
    hidden_outputs = np.array(hidden_outputs).reshape(20, 8)
    # print(np.shape(hidden_outputs))
    
    backward_propagation(inputs, outputs, hidden_outputs, hidden_weights, input_weights, hidden_bias, output_bias)

# Train the neural network
# learning_rate = 0.1
# epochs = 2000
# train(inputs, learning_rate, epochs)


[168600.62933249]
[166994.59035467]
[166994.59035467]
[166994.59035467]
[166994.59035467]
[166994.59035467]
[166994.59035467]
[166994.59035467]
[166994.59035467]
[166994.59035467]
[166994.59035467]
[166994.59035467]
[166994.59035467]
[166994.59035467]
[166994.59035467]
[166994.59035467]
[166994.59035467]
[166994.59035467]
[166994.59035467]
[166994.59035467]


In [16]:
initial_input = np.array([[0], [0], [np.pi/6], [0]])
inputs = [initial_input]

initial_output, hidden_layer =  forward_propagation(initial_input)
outputs = [initial_output]
hidden_outputs = [hidden_layer]

for i in range(10000):
    x_curr = inputs[i]
    u_curr = outputs[i]
    x_next = next_state(A, B, x_curr, u_curr, dt = 0.001)
    inputs.append(x_next)
    output_layer, hidden_layer = forward_propagation(x_next)
    outputs.append(output_layer)
    hidden_outputs.append(hidden_layer)

print(inputs)

[array([[0.        ],
       [0.        ],
       [0.52359878],
       [0.        ]]), array([[ 0.00000000e+00],
       [-7.82817000e-05],
       [ 5.23598776e-01],
       [ 1.04190994e-02]]), array([[ 7.82817000e-08],
       [-1.56563400e-04],
       [ 5.23609195e-01],
       [ 2.08381988e-02]]), array([[ 2.34845100e-07],
       [-2.34847653e-04],
       [ 5.23630033e-01],
       [ 3.12575075e-02]]), array([[ 4.69692753e-07],
       [-3.13137011e-04],
       [ 5.23661290e-01],
       [ 4.16772349e-02]]), array([[ 7.82829764e-07],
       [-3.91434027e-04],
       [ 5.23702968e-01],
       [ 5.20975902e-02]]), array([[ 1.17426379e-06],
       [-4.69741254e-04],
       [ 5.23755065e-01],
       [ 6.25187828e-02]]), array([[ 1.64400504e-06],
       [-5.48061245e-04],
       [ 5.23817584e-01],
       [ 7.29410221e-02]]), array([[ 2.19206629e-06],
       [-6.26396553e-04],
       [ 5.23890525e-01],
       [ 8.33645174e-02]]), array([[ 2.81846284e-06],
       [-7.04749732e-04],
       [ 5.23

In [24]:
np.shape(inputs)[0]

1000

In [2]:
# train network
for i in range(1000):
    train(A, np.array([10.6, 0, 0, 0]))



Iteration: 0, Loss: 1.3462765216827393
Iteration: 100, Loss: 0.836516261100769
Iteration: 200, Loss: 0.8298045992851257
Iteration: 300, Loss: 0.8285586833953857
Iteration: 400, Loss: 0.8283164501190186
Iteration: 500, Loss: 0.8282677531242371
Iteration: 600, Loss: 0.828260064125061
Iteration: 700, Loss: 0.8282592296600342
Iteration: 800, Loss: 0.8282591104507446
Iteration: 900, Loss: 0.8282590508460999
